In [1]:
import pandas as pd
import numpy as np

In [33]:
# read in turnstile data from the week of September 19, 2019
cols = ['control_area','unit','scp','station','line_name','division','date','time',
        'desc','entries','exits']
turnstile_data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190921.txt',
                            header = 0, names = cols)

In [38]:
#convert date and time to DateTime format in a single column
turnstile_data['concat_date_time'] = turnstile_data['date'] + ' ' + turnstile_data['time']
turnstile_data['date_time'] = pd.to_datetime(turnstile_data.concat_date_time,format = '%m/%d/%Y %H:%M:%S')

#delete unformatted date and time columns
del turnstile_data['concat_date_time']
del turnstile_data['date']
del turnstile_data['time']

turnstile_data.head(20)

KeyError: 'date'

378